## TODO

- [ ] FHIR-Daten abfragen
  - [x] ResearchStudy
  - [x] ResearchSubject
  - [x] Patient
  - [x] Procedure
  - [x] Condition
- [x] CSV cleanen
- [x] CSV in Material laden
- [x] Laden der CSV testen
- [ ] Laden der CSV einbauen
- [ ] Mergen FHIR mit CSV

# Block 1: Medizininformatik

Eure Ansprechpartner sind Hannes Ulrich und Joshua Wiedekopf.

In diesem Block geht es darum, die Daten aus der TCGA-Studie aus disjunkten Quellen in ein Format zu bringen, mit dem in den folgenden Blöcken weiter gearbeitet werden kann.

Hierzu werden wir auf einen FHIR-Server zugreifen, um die Patientendaten abzurufen, während die klinischen Daten im CSV-Format vorliegen.

## Pakete laden und installieren

**Dieser Block muss einmal unbedingt ausgeführt werden, damit in Eurer Runtime die notwendigen Pakete zur Verfügung stehen!**

In [ ]:
install.packages("fhircrackr")
install.packages("assertthat")
install.packages("assertr")

Sollten Konflikte gemeldet werden, ist dies in der Regel in Ordnung.

In [1]:
library(assertthat)
library(fhircrackr)
library(tidyverse)
library(glue)
library(assertr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()    masks stats::filter()
✖ tibble::has_name() masks assertthat::has_name()
✖ dplyr::lag()       masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
#@title Hilfsfunktionen, einfach ignorieren, nur einmal ausführen
fill_me <- function(name, description) {
  error_msg = glue("Die Variable/der Parameter '{name}' ist nicht gefüllt: {description}")
  stop(error_msg)
}
fun_me <- function(description) {
  error_msg = glue("Hier fehlt ein Funktionsaufruf: {description}")
  stop(error_msg)
}
test <- function(count=1, fun) {
  fun()
  print(ifelse(count == 1, "1 Test erfolgreich", str_glue("{count} Tests erfolgreich")))
}


## Daten laden vom FHIR-Server

Habt ihr Probleme bei der Umsetzung dieser Aufgabe, ist die [Dokumentation von HL7 FHIR 🇬🇧](https://hl7.org/fhir/R5/) hilfreich. Insbesondere die folgenden Seiten können wichtig sein:
- [FHIR Search](https://hl7.org/fhir/R5/search.html)
- [Ressource *Patient*](https://hl7.org/fhir/R5/patient.html)
- [Ressource *Condition*](https://hl7.org/fhir/R5/condition.html)
- [Ressource *Observation*](https://hl7.org/fhir/R5/observation.html)
- [Ressource *ResearchStudy*](https://hl7.org/fhir/R5/researchstudy.html)
- [Ressource *ResearchSubject*](https://hl7.org/fhir/R5/researchsubject.html)

Darüber hinaus ist die [Dokumentation vom `fhircrackr`-R-Paket 🇬🇧](https://cran.r-project.org/web/packages/fhircrackr) sehr hilfreich:
- [Download FHIR resources](https://cran.r-project.org/web/packages/fhircrackr/vignettes/downloadResources.html)


In [4]:
endpoint="https://fhir.imi.uni-luebeck.de/fhir"
# die Adresse unseres FHIR-Servers. Unter https://fhir.imi.uni-luebeck.de/ ist auch eine einfache GUI verfügbar

## Aufgabe 1

Ladet alle Ressourcen vom Typ `ResearchStudy` mit folgendem Identifier:
- `system` = `https://www.cbioportal.org/study`
- `value`  = `paad_tcga_pan_can_atlas_2018`

Dabei sollte genau eine Ressource gefunden werden.

In [5]:
request <- fhir_url(
  url = endpoint,
  resource = fill_me("resource", "Hier soll der ResourceType, den wir suchen, rein"),
  parameters = list(
    fill_me("parameters", "Hier können Suchparameter angegeben werden.")
  )
)
studies_bundle <- fun_me("Die Suchanfrage muss noch an den Server geschickt werden")
str(studies_bundle) # Gibt die Datenstruktur aus, mit der wir arbeiten

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'resource' in selecting a method for function 'fhir_url': Die Variable/der Parameter 'resource' ist nicht gefüllt: Hier soll der ResourceType, den wir suchen, rein


In [6]:
#@title Lösung
request <- fhir_url(
  url = endpoint,
  resource = "ResearchStudy",
  parameters = list(
    "identifier" = "https://www.cbioportal.org/study|paad_tcga_pan_can_atlas_2018"
  )
)
studies_bundle <- fhir_search(request, verbose=0)
str(studies_bundle)

Formal class 'fhir_bundle_list' [package "fhircrackr"] with 1 slot
  ..@ .Data:List of 1
  .. ..$ :Formal class 'fhir_bundle_xml' [package "fhircrackr"] with 3 slots
  .. .. .. ..@ next_link:Formal class 'fhir_url' [package "fhircrackr"] with 1 slot
  .. .. .. .. .. ..@ .Data: chr(0) 
  .. .. .. ..@ self_link:Formal class 'fhir_url' [package "fhircrackr"] with 1 slot
  .. .. .. .. .. ..@ .Data: chr "https://fhir.imi.uni-luebeck.de/fhir/ResearchStudy?identifier=https%3A%2F%2Fwww.cbioportal.org%2Fstudy%7Cpaad_t"| __truncated__
  .. .. .. ..@ .S3Class : chr [1:2] "xml_document" "xml_node"
  .. .. .. ..$ names: chr [1:2] "node" "doc"


In [7]:
#@title Test der Lösung
test(count=2, fun=function() {
  assert_that(length(studies_bundle) == 1)
  assert_that(is(studies_bundle, "fhir_bundle_list"))
})

[1] "2 Tests erfolgreich"


## Aufgabe 2

Gut, jetzt haben wir ein FHIR-Bundle. Damit wir innerhalb von R damit auch arbeiten können, verwenden wir die `fhir_crack`-Funktion von `fhircrackr`, um die Daten in die R-native Datenstruktur `dataframe` zu bekommen (bzw. vielmehr ein `tibble`, die ["neuere, bessere" Alternative zum `dataframe`](https://stackoverflow.com/questions/64856424/what-are-the-differences-between-data-frame-tibble-and-matrix)).

Das "Flachklopfen" der FHIR-Ressourcen ist in der Dokumentation von `fhircrackr` [beschrieben](https://cran.r-project.org/web/packages/fhircrackr/vignettes/fhircrackr_intro.html#flattening-resources).

Es muss also eine Tabellen-Definition erstellt werden und dann diese Definition auf unser Bundle angewendet werden. Uns interessiert nicht alles, sogar fast das meiste nicht. Daher können wir die Spaltendefinitionen relativ knapp fassen. Damit FHIR-Referenzen aufgelöst werden können, benötigen wir die technische `id` der ResearchStudy. Damit wir selber mit dem DataFrame etwas anfangen können, geben wir noch den `title` und die Komponenten des `identifier` aus.

In [8]:
#@title Lösung
table_description_studies <- fhir_table_description(
  resource="ResearchStudy",
  cols=c(
    id="id",
    title="title",
    identifier_system="identifier/system",
    identifier_value="identifier/value"
  )
)
studies <- as_tibble(fhir_crack(bundles=studies_bundle, design=table_description_studies, verbose=0))
studies
study_id = (studies$id)[1]

id,title,identifier_system,identifier_value
<chr>,<chr>,<chr>,<chr>
1,"Pancreatic Adenocarcinoma (TCGA, PanCancer Atlas)",https://www.cbioportal.org/study,paad_tcga_pan_can_atlas_2018


In [34]:
#@title Test der Lösung
test(count=2, fun=function() {
  studies |>
    verify(has_all_names("id", "title")) |>
    verify(identifier_value == 'paad_tcga_pan_can_atlas_2018')
})

[1] "2 Tests erfolgreich"


## Aufgabe 3

Wir haben jetzt eine Referenz auf die ResearchStudy, sodass wir auf Basis dieser nach den inkludierten Patienten suchen können - andere Patienten wollen wir natürlich nicht abfragen, weil wir es nicht dürfen 😎.

Die Teilnahme eines Patienten ist an die Ressource `ResearchSubject` geknüpft, die Brücke zwischen `ResearchStudy` und `Patient`. In `ResearchSubject` wird eine studieninterne ID zugewiesen, die wir benötigen, in `Patient` ist neben `gender` und `deceased` auch die ID relevant, damit wir über diese die Prozedur- und Diagnose-Ressourcen im nächsten Schritt anfragen können.

Wir werden also die `ResearchSubject`-Ressourcen, die auf unsere `ResearchStudy` verweisen, anfragen. Danach fragen wir die Patienten an, die in `ResearchSubject` referenziert werden.

Zur Datenmanipulation sind Pipes `|>` und das Paket `dyplr` sehr hilfreich:

```r
data |>
  filter(x='foo') |>
  mutate(y=x+3)
```

Mehr Informationen dazu im [Data Wrangling Cheat Sheet](https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf) von `dplyr`.

In [25]:
#@title Lösung
subject_patient_url <- fhir_url(
  url = endpoint,
  resource = "ResearchSubject",
  parameters = list(
    "study"=glue("ResearchStudy/{study_id}"),
    "_include"="ResearchSubject:patient"
  )
)

print(subject_patient_url)

subject_bundle <- fhir_search(subject_patient_url, verbose=0)
res_subject_design <- fhir_table_description(
  resource = "ResearchSubject",
  cols=c(
    subject_id="id",
    patient_id="subject/reference"
  )
)
pat_design <- fhir_table_description(
  resource = "Patient",
  cols=c(
    patient_id="id",
    deceasedBoolean="deceasedBoolean",
    gender="gender"
  )
)

combined_design <- fhir_design(ResearchSubject=res_subject_design, Patient=pat_design)
data <- fhir_crack(bundles=subject_bundle, design=combined_design, verbose=0)

# Reduktion auf die nötigsten Daten und leichte Manipulationen in einer Spalte (um einen einfacheren Join zu machen)
research_subject_data <- data$ResearchSubject |>
  mutate(patient_id=gsub("Patient/", "", patient_id))
patient_data <- data$Patient

joined_subject_data <- research_subject_data |>
  left_join(patient_data, by="patient_id")
head(joined_subject_data)

An object of class "fhir_url"
[1] "https://fhir.imi.uni-luebeck.de/fhir/ResearchSubject?study=ResearchStudy/1&_include=ResearchSubject:patient"


,subject_id,patient_id,deceasedBoolean,gender
,<chr>,<chr>,<chr>,<chr>
1,4,3,false,male
2,7,6,true,male
3,10,9,false,male
4,13,12,true,male
5,17,16,false,female
6,20,19,false,male


In [30]:
#@title Test der Lösung

test(count=3, fun=function() {
  joined_subject_data |>
    verify(has_all_names("subject_id", "patient_id", "deceasedBoolean", "gender")) |>
    verify(any(patient_id == 12)) |>
    verify(any(subject_id == 20))
})

[1] "3 Tests erfolgreich"


## Aufgabe 4



Hier: Prozedur- und Diagnosedaten anfragen

In [33]:
patient_id_join <- paste(joined_subject_data$patient_id, collapse = ',')
condition_url <- fhir_url(
  url=endpoint,
  resource="Condition",
  parameters=c(
    "subject"=patient_id_join
  )
)
condition_data <- fhir_crack(
  bundles=fhir_search(condition_url, verbose=0),
  design=fhir_table_description(
    resource="Condition",
    cols=c(
      condition_id="id",
      condition_code="code/coding/code",
      condition_display="code/coding/display",
      condition_onsetAge_unit="onsetAge/unit",
      condition_onsetAge_value="onsetAge/value",
      patient_id="subject/reference"
    )
  )
)
condition_data <- condition_data |>
  mutate(patient_id = gsub("Patient/", "", patient_id))

procedure_url <- fhir_url(
  url=endpoint,
  resource="Procedure",
  parameters=c(
    subject=patient_id_join
  )
)
procedure_data <- fhir_crack(
  bundles=fhir_search(procedure_url, verbose=0),
  design=fhir_table_description(
    resource="Procedure",
    cols=c(
      "procedure_code"="code/coding/code",
      "procedure_display"="code/coding/display",
      "procedure_system"="code/coding/system",
      "procedure_id"="id",
      "patient_id"="subject/reference"
    )
  )
) |>
  mutate(patient_id = gsub("Patient/", "", patient_id))

joined_clinical_data <- joined_subject_data |>
  left_join(condition_data, by="patient_id") |>
  left_join(procedure_data, by="patient_id")
joined_clinical_data

Cracking 10 Conditions' Bundles on a LINUX-Engine using 1/16 CPUs ... 
finished.
Cracking 3 Procedures' Bundles on a LINUX-Engine using 1/16 CPUs ... 
finished.


subject_id,patient_id,deceasedBoolean,gender,condition_id,condition_code,condition_display,condition_onsetAge_unit,condition_onsetAge_value,procedure_code,procedure_display,procedure_system,procedure_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4,3,false,male,2,C25.0,Bösartige Neubildung: Pankreaskopf,a,65.0,NA,NA,NA,NA
7,6,true,male,5,C25.8,"Bösartige Neubildung: Pankreas, mehrere Teilbereiche überlappend",a,48.0,NA,NA,NA,NA
10,9,false,male,8,C25.1,Bösartige Neubildung: Pankreaskörper,a,75.0,NA,NA,NA,NA
13,12,true,male,11,C25.0,Bösartige Neubildung: Pankreaskopf,a,71.0,NA,NA,NA,NA
17,16,false,female,14,C25.0,Bösartige Neubildung: Pankreaskopf,a,70.0,1287742003,Radiotherapy (procedure),http://snomed.info/sct,15
20,19,false,male,18,C25.0,Bösartige Neubildung: Pankreaskopf,a,55.0,NA,NA,NA,NA
24,23,true,male,21,C25.1,Bösartige Neubildung: Pankreaskörper,a,73.0,1287742003,Radiotherapy (procedure),http://snomed.info/sct,22
28,27,false,male,25,C25.0,Bösartige Neubildung: Pankreaskopf,a,73.0,1287742003,Radiotherapy (procedure),http://snomed.info/sct,26
31,30,true,male,29,C25.1,Bösartige Neubildung: Pankreaskörper,a,61.0,NA,NA,NA,NA
